In [1]:
import os
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

path = "./Dataset"  # 데이터셋이 저장된 경로
file_list = os.listdir(path)    # 데이터셋의 목록 읽어오기

actions = []    # 파일 확장자 빼고 제스처 이름만 저장
for i in range(len(file_list)) :
    actions.append(file_list[i][:-4])

seq_length = 10

In [2]:
# 제스처별로 전처리하고 훈련데이터 시험데이터 생성하기
datax, datay = [], []

# for i in range(1):
for i in range(len(file_list)):

    # 데이터프레임 읽고 변수, 라벨 분리
    df = pd.read_csv(f'{path}/{file_list[i]}', header=None)
    dfx = df.iloc[:, :-1].to_numpy()
    dfy = df.iloc[:, -1].to_numpy()

    # 시퀀스길이에 따라 시퀀스데이터로 변환
    for i in range(len(df) - seq_length):
        datax.append(dfx[i: i + seq_length])
        datay.append(dfy[i + seq_length])

In [3]:
# # 정답지 원-핫 인코딩
# datay = to_categorical(datay, num_classes=len(actions))

# 문제지 정답지 타입 변환
datax = np.array(datax)
datay = np.array(datay)

In [4]:
# 훈련데이터 시험데이터 분리
xtrain, xtest, ytrain, ytest = train_test_split(datax, datay, test_size=0.1)

In [5]:
from tensorflow import keras

model = keras.Sequential()
model.add( keras.layers.LSTM(64, activation = 'relu', input_shape = xtrain.shape[1:] ))
model.add( keras.layers.Dense(32, activation='relu') )
model.add( keras.layers.Dense(len(actions), activation='softmax') )

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = 'accuracy')                              # sparse categorical crossentropy는 정수, categorical crossentropy는 원핫인코딩

checkpoint = keras.callbacks.ModelCheckpoint('./Models/model.h5', verbose=1, save_best_only=True, mode='auto')
reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto')
early = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.fit(  xtrain, ytrain,
            validation_data = (xtest, ytest),
            epochs = 100,
            callbacks = [checkpoint, reduceLR, early]
            )

Epoch 1/100
 1/11 [=>............................] - ETA: 8s - loss: 13.0560 - accuracy: 0.0938
Epoch 1: val_loss improved from inf to 2.23242, saving model to ./Models\model.h5
11/11 [==============================] - 1s 26ms/step - loss: 4.1922 - accuracy: 0.4678 - val_loss: 2.2324 - val_accuracy: 0.7436 - lr: 0.0010
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 0.6621 - accuracy: 0.9062
Epoch 2: val_loss improved from 2.23242 to 1.03271, saving model to ./Models\model.h5
11/11 [==============================] - 0s 7ms/step - loss: 0.7676 - accuracy: 0.8567 - val_loss: 1.0327 - val_accuracy: 0.8462 - lr: 0.0010
Epoch 3/100
 1/11 [=>............................] - ETA: 0s - loss: 0.9928 - accuracy: 0.8438
Epoch 3: val_loss improved from 1.03271 to 0.09238, saving model to ./Models\model.h5
11/11 [==============================] - 0s 7ms/step - loss: 0.2077 - accuracy: 0.9591 - val_loss: 0.0924 - val_accuracy: 0.9744 - lr: 0.0010
Epoch 4/100
 1/11 [=>............